In [2]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

from sklearn.utils import shuffle
from sklearn.neighbors import NearestCentroid, KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import seaborn as sns

from keras.datasets import fashion_mnist
from keras.utils.np_utils import to_categorical   

import csv
import os

import warnings
warnings.filterwarnings('ignore')

seed = 1234
np.random.seed(seed) 

In [5]:
## Image clean source: https://www.kaggle.com/code/uzairrj/beg-tut-intel-image-classification-93-76-accur

def get_images(path):
    IntelData, Labels = [], []
    
    for labels in os.listdir(path):
        for image_file in os.listdir(path+labels):
            image = cv2.imread(path+labels+r'/'+image_file)
            image = cv2.resize(image,(150,150))
            IntelData.append(image)
            Labels.append(Word_to_Label[labels])
    return shuffle(Images,Labels,random_state=seed)
    
def Label_to_Word(label):
    label_dict = {0:"buildings", 1:"forest", 2:"glacier", 3:"mountain", 4:"sea", 5:"street"}
    return label_dict[label]

def Word_to_Label(word):
    word_dict = {"building": 0, "forest":1, "glacier":2, "mountain":3, "sea":4, "street":5}
    return word_dict[word]